# Multiple Chromosomes Simulation

In [1]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from openmm.app import *

In [2]:
block = 5*10**2
n_blocks = 1*10**3

**SYSTEM SIM**  
this is done after setting up the collapse simulations of both chromosome 10 and 11 separately  

In [3]:
# once the collapse simulations are done, the collapsed structures of each chromosome should be included in the same simulation system
# SETTING UP SIMULATION VARIABLE INIT
sim_chr10_chr11 = MiChroM(name="chr10_chr11", temperature=1.0, time_step=0.01)
sim_chr10_chr11.setup(platform="cuda")
sim_chr10_chr11.saveFolder('multi_op_chr10_11_allFrames')


# The function loadNDB receives a list of files and saves the chromosome collapse structures in the variable Struc_chr10_chr11.
struc_chr10_chr11 = sim_chr10_chr11.loadNDB(NDBfiles=[
    'multi_op_chr10_11_allFrames/chr10_0_block1000.ndb', 
    'multi_op_chr10_11_allFrames/chr11_0_block1000.ndb'
])

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.5 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

**Chains:  [(0, 2711, 0), (2712, 5414, 0)]**  
* mol_1 from idx = 0 &rarr; idx = 2711  
* mol_2 from idx = 2712 &rarr; idx = 5414  
___  

In [4]:
# distribute the chromosome chains
# essential for randomizing the initial condition when simulating different replicas

# Struc_chr10_chr11 = sim_chr10_chr11.setFibPosition(Struc_chr10_chr11, dist=(1.5,3.0))
# 'dist' PARAMETER IS NOT DEFINED IN THE LIBRARY

struc_chr10_chr11 = sim_chr10_chr11.setFibPosition(struc_chr10_chr11)
sim_chr10_chr11.loadStructure(struc_chr10_chr11, center=True)
sim_chr10_chr11.saveStructure(mode='ndb')
sim_chr10_chr11.saveStructure(mode='pdb')

In [5]:
# ADDING FORCES TO THE SIM STRUCTURES
sim_chr10_chr11.addFENEBonds(kfb=30.0)
sim_chr10_chr11.addAngles(ka=2.0)
sim_chr10_chr11.addRepulsiveSoftCore(Ecut=4.0)
sim_chr10_chr11.addFlatBottomHarmonic(n_rad=20)

# The addTypetoType interaction is independent of the chromosome chain and only depends on the loci types. This potential is added for both chains simultaneously.
sim_chr10_chr11.addTypetoType(mu=3.22, rc = 1.78)

# The function addMultiChainIC receives the chromosome chain and adds the IC potential.
sim_chr10_chr11.addMultiChainIC(chainIndex=0, mu=3.22, rc = 1.78, dinit=3, dend=500)
sim_chr10_chr11.addMultiChainIC(chainIndex=1, mu=3.22, rc = 1.78, dinit=3, dend=500)

In [6]:
# run the production simulation
sim_chr10_chr11.initStorage('traj_chr10_chr11_allFrames', mode='w')
pdb = PDBFile(
    'multi_op_chr10_11_allFrames/chr10_chr11_0_block0.pdb',
    'multi_op_chr10_11_allFrames/chr10_chr11_1_block0.pdb'
) # check file path

top = pdb.getTopology()
top_file = 'multi_top_allFrames.dcd'

with open(top_file, 'wb') as f:
    dcd = DCDFile(f, top, 0.01)
    print('RUNNING SIMULATION', '\n---\n')
    for _ in range(n_blocks):
        sim_chr10_chr11.runSimBlock(block) 
        sim_chr10_chr11.saveStructure()
        sim_chr10_chr11.saveStructure(mode='pdb')
        dcd.writeModel(sim_chr10_chr11.getPositions())
        print('3rd loop', '\n---\n')

sim_chr10_chr11.storage[0].close() #close the cndb file for chr10
sim_chr10_chr11.storage[1].close() #close the cndb file for chr11
sim_chr10_chr11.saveStructure(mode="ndb")
sim_chr10_chr11.saveStructure(mode="pdb")

RUNNING SIMULATION 
---

Number of exceptions: 5413
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force
adding force  RepulsiveSoftCore 2
adding force  FlatBottomHarmonic 3
Add exclusions for TypetoType force
adding force  TypetoType 4
Add exclusions for IdealChromosomeChain0 force
adding force  IdealChromosomeChain0 5
Add exclusions for IdealChromosomeChain1 force
adding force  IdealChromosomeChain1 6
Positions... 
 loaded!
potential energy is 19.788648
bl=1 pos[1]=[-5.5 -9.7 25.1] dr=0.93 t=0.0ps kin=1.50 pot=19.81 Rg=20.228 SPS=1861 


ValueError: The number of positions must match the number of atoms

**ERROR**  
previous cell gives this error &rarr; `ValueError: The number of positions must match the number of atoms`  
at this line &rarr; `dcd.writeModel(sim_chr10_chr11.getPositions())`  
___  

**checking a few things**  

In [7]:
# TESTING WITH loadNDB
struc_chr10_chr11 = sim_chr10_chr11.loadNDB(NDBfiles=[
    'multi_op_chr10_11_allFrames/chr10_0_block1000.ndb', 
    'multi_op_chr10_11_allFrames/chr11_0_block1000.ndb'
])
print('with loadNDB():')
print(struc_chr10_chr11.shape)
print(type(struc_chr10_chr11))
print(struc_chr10_chr11)

Chains:  [(0, 2711, 0), (2712, 5414, 0)]
with loadNDB():
(5415, 3)
<class 'numpy.ndarray'>
[[ -2.466  -2.271  11.98 ]
 [ -1.975  -3.083  11.722]
 [ -1.353  -3.381  11.076]
 ...
 [  6.258 -10.274  -0.412]
 [  5.321 -10.233  -0.633]
 [  4.458 -10.783  -0.787]]


In [8]:
# TESTING WITH loadPDB
struc_chr10_chr11 = sim_chr10_chr11.loadPDB(PDBfiles=[
    'multi_op_chr10_11_allFrames/chr10_0_block1000.pdb', 
    'multi_op_chr10_11_allFrames/chr11_0_block1000.pdb'
])
print('with loadPDB():')
print(struc_chr10_chr11.shape)
print(type(struc_chr10_chr11))
print(struc_chr10_chr11)

Chains:  [(0, 2711, 0), (2712, 5414, 0)]
with loadPDB():
(5415, 3)
<class 'numpy.ndarray'>
[[ -2.466  -2.271  11.98 ]
 [ -1.975  -3.083  11.722]
 [ -1.353  -3.381  11.076]
 ...
 [  6.258 -10.274  -0.412]
 [  5.321 -10.233  -0.633]
 [  4.458 -10.783  -0.787]]


---  
so now we know that both file loaders load the same values with the same structures  

In [9]:
print(type(sim_chr10_chr11))
print(sim_chr10_chr11)

<class 'OpenMiChroM.ChromDynamics.MiChroM'>


`struc_chr10_chr11` &rarr; array of molecule structure  
`sim_chr10_chr11` &rarr; simulation variable &rarr; loads variable of array structure  

In [10]:
positions = sim_chr10_chr11.getPositions()
print('concerning sim_chr10_chr11:')
print(type(positions))
print(positions.shape)
print(positions)

concerning sim_chr10_chr11:
<class 'numpy.ndarray'>
(5415, 3)
[[ -5.4840245   -9.707231    25.119858  ]
 [ -5.335743   -10.554476    25.484241  ]
 [ -5.1677423  -11.397454    25.014368  ]
 ...
 [  8.778797     0.43869022 -17.10517   ]
 [  8.29467      0.42137846 -16.229094  ]
 [  7.625365     0.8336093  -15.681841  ]]


**Remember :** `Chains` end at idx 5414 &rarr; `shape[0]` = 5415

In [11]:
pdb = PDBFile(
    'multi_op_chr10_11_allFrames/chr10_chr11_0_block0.pdb',
    'multi_op_chr10_11_allFrames/chr10_chr11_1_block0.pdb'
)
print('concerning PDBFile method:')
print(pdb)
print(len(pdb.getPositions()))

concerning PDBFile method:
2712


**Notice :**  
* pdb variable has `shape[0]` = 2712 &rarr; only reading first file  
* `loadPDB` &rarr; returns array  
* `PDBFile` &rarr; returns PDBFile object  

**POSSIBLE SOLUTIONS :**  
1. read each file separately as `.pdb` &rarr; concatenate both files as `.pdb`  
2. read both files together as arrays &rarr; convert array to pdb object  
___  

**FIRST APPROACH**  
read each file separately as `.pdb` &rarr; concatenate both files as `.pdb`  

In [12]:
pdb_10 = PDBFile('multi_op_chr10_11_allFrames/chr10_chr11_0_block0.pdb')
pdb_11 = PDBFile('multi_op_chr10_11_allFrames/chr10_chr11_1_block0.pdb')
print('pdb_10 length = ', len(pdb_10.getPositions()))
print('pdb_11 length = ', len(pdb_11.getPositions()))

temp_arr = np.array(pdb_10.getPositions())
print('temp_arr shape = ', temp_arr.shape)
# temp_pdb = PDBFile(temp_arr)    # 'numpy.ndarray' object has no attribute 'decode'

top_10 = pdb_10.getTopology()
top_11 = pdb_11.getTopology()
print('top_10 type = ', type(top_10))
print(top_10.chains)

pdb_10 length =  2712
pdb_11 length =  2703
temp_arr shape =  (2712, 3)
top_10 type =  <class 'openmm.app.topology.Topology'>
<bound method Topology.chains of <Topology; 1 chains, 2712 residues, 2712 atoms, 0 bonds>>


___

In [13]:
from Bio import PDB

pdb_io = PDB.PDBIO()

structures = ['multi_op_chr10_11_allFrames/chr10_chr11_0_block0.pdb','multi_op_chr10_11_allFrames/chr10_chr11_1_block0.pdb']

pdb_file = 'all_struc.pdb'
with pdb_file as open_file:
    for struct in structures:
        pdb_io.set_structure(struct)
        pdb_io.save(open_file)

AttributeError: __enter__

&uarr; undiagnosed error  
___

In [14]:
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
parser = PDBParser()
structure = parser.get_structure("ch10", "multi_op_chr10_11_allFrames/chr10_chr11_0_block0.pdb")
structure2 = parser.get_structure("ch11", "multi_op_chr10_11_allFrames/chr10_chr11_1_block0.pdb")

io=PDBIO()
io.set_structure(structure)
io.save("bio-pdb-pdbio-out.pdb")
io.set_structure(structure2)
io.save("bio-pdb-pdbio-out.pdb")

&uarr; overwrites exsiting file  
___

**when trying to combine `.pdb` files with _Chimera_**  

In [15]:
test_pdb = PDBFile('multi_op_chr10_11_allFrames/comb.pdb')
test_pos = test_pdb.getPositions()
len(test_pos)

2712

**Notice :** _Chimera_'s saved file has a similar issue of saving only the first file  
___

**when trying to combine `.pdb` files with _PyMOL_**  

In [9]:
test_pdb = PDBFile('multi_op_chr10_11_allFrames/chr10_chr11_combined_pymol.pdb')
test_pos = test_pdb.getPositions()
len(test_pos)

5415

**Notice :** _PyMOL_'s saved file has successfully combined both files into one **BUT** with the wrong dimensions  
___  

**SECOND APPROACH**  
read both files together as arrays &rarr; convert array to pdb object  

___

**Testing production simulation with newly generated `.pdb` file**

In [10]:
sim_chr10_chr11 = MiChroM(name="chr10_chr11", temperature=1.0, time_step=0.01)
sim_chr10_chr11.setup(platform="cuda")
sim_chr10_chr11.saveFolder('multi_op_chr10_11_allFrames')
struc_chr10_chr11 = sim_chr10_chr11.loadNDB(NDBfiles=[
    'multi_op_chr10_11_allFrames/chr10_0_block1000.ndb', 
    'multi_op_chr10_11_allFrames/chr11_0_block1000.ndb'
])

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.5 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

In [11]:
struc_chr10_chr11 = sim_chr10_chr11.setFibPosition(struc_chr10_chr11)
sim_chr10_chr11.loadStructure(struc_chr10_chr11, center=True)
sim_chr10_chr11.saveStructure(mode='ndb')
sim_chr10_chr11.saveStructure(mode='pdb')

In [12]:
sim_chr10_chr11.addFENEBonds(kfb=30.0)
sim_chr10_chr11.addAngles(ka=2.0)
sim_chr10_chr11.addRepulsiveSoftCore(Ecut=4.0)
sim_chr10_chr11.addFlatBottomHarmonic(n_rad=20)
sim_chr10_chr11.addTypetoType(mu=3.22, rc = 1.78)
sim_chr10_chr11.addMultiChainIC(chainIndex=0, mu=3.22, rc = 1.78, dinit=3, dend=500)
sim_chr10_chr11.addMultiChainIC(chainIndex=1, mu=3.22, rc = 1.78, dinit=3, dend=500)

In [13]:
# run the production simulation
sim_chr10_chr11.initStorage('traj_chr10_chr11_allFrames', mode='w')
pdb = PDBFile('multi_op_chr10_11_allFrames/chr10_chr11_combined_pymol.pdb')

top = pdb.getTopology()
top_file = 'multi_top_allFrames.dcd'

with open(top_file, 'wb') as f:
    dcd = DCDFile(f, top, 0.01)
    print('RUNNING SIMULATION', '\n---\n')
    for _ in range(n_blocks):
        sim_chr10_chr11.runSimBlock(block) 
        sim_chr10_chr11.saveStructure()
        sim_chr10_chr11.saveStructure(mode='pdb')
        dcd.writeModel(sim_chr10_chr11.getPositions())
        print('3rd loop', '\n---\n')

sim_chr10_chr11.storage[0].close() #close the cndb file for chr10
sim_chr10_chr11.storage[1].close() #close the cndb file for chr11
sim_chr10_chr11.saveStructure(mode="ndb")
sim_chr10_chr11.saveStructure(mode="pdb")

RUNNING SIMULATION 
---

Number of exceptions: 5413
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force
adding force  RepulsiveSoftCore 2
adding force  FlatBottomHarmonic 3
Add exclusions for TypetoType force
adding force  TypetoType 4
Add exclusions for IdealChromosomeChain0 force
adding force  IdealChromosomeChain0 5
Add exclusions for IdealChromosomeChain1 force
adding force  IdealChromosomeChain1 6
Positions... 
 loaded!
potential energy is 19.779928
bl=1 pos[1]=[-14.9 -11.3 3.8] dr=0.96 t=0.0ps kin=1.55 pot=19.75 Rg=19.427 SPS=1929 
3rd loop 
---

bl=2 pos[1]=[-14.9 -12.2 4.6] dr=0.94 t=0.0ps kin=1.51 pot=19.80 Rg=19.419 SPS=2166 
3rd loop 
---

bl=3 pos[1]=[-17.9 -14.2 4.4] dr=0.95 t=0.0ps kin=1.53 pot=19.77 Rg=19.400 SPS=2131 
3rd loop 
---

bl=4 pos[1]=[-15.8 -12.8 4.2] dr=0.95 t=0.0ps kin=1.50 pot=19.78 Rg=19.393 SPS=2222 
3rd loop 
---

bl=5 pos[1]=[-16.9 -13.6 3.9] dr=0.95 t=0.0ps kin=1.47 pot=19.80 Rg=19.376 SPS=2252 
3rd loop 
---

bl=136 pos[1]=[-6.2 -12.5 -3.1] dr=0.91 t=0.0ps kin=1.50 pot=19.72 Rg=16.923 SPS=2175 
3rd loop 
---

bl=137 pos[1]=[-7.0 -12.9 -1.9] dr=0.93 t=0.0ps kin=1.47 pot=19.75 Rg=16.917 SPS=2123 
3rd loop 
---

bl=138 pos[1]=[-7.1 -12.5 -2.1] dr=0.93 t=0.0ps kin=1.49 pot=19.74 Rg=16.911 SPS=2291 
3rd loop 
---

bl=139 pos[1]=[-8.1 -12.2 -2.2] dr=0.94 t=0.0ps kin=1.49 pot=19.78 Rg=16.891 SPS=2276 
3rd loop 
---

bl=140 pos[1]=[-7.7 -11.8 -2.6] dr=0.93 t=0.0ps kin=1.51 pot=19.78 Rg=16.861 SPS=2229 
3rd loop 
---

bl=141 pos[1]=[-7.4 -12.5 -2.8] dr=0.94 t=0.0ps kin=1.52 pot=19.79 Rg=16.867 SPS=2186 
3rd loop 
---

bl=142 pos[1]=[-7.0 -14.0 -3.6] dr=0.94 t=0.0ps kin=1.52 pot=19.79 Rg=16.877 SPS=2311 
3rd loop 
---

bl=143 pos[1]=[-7.0 -14.0 -3.6] dr=0.95 t=0.0ps kin=1.52 pot=19.79 Rg=16.902 SPS=2234 
3rd loop 
---

bl=144 pos[1]=[-6.5 -13.4 -3.6] dr=0.92 t=0.0ps kin=1.49 pot=19.80 Rg=16.908 SPS=2253 
3rd loop 
---

bl=145 pos[1]=[-6.9 -14.5 -2.5] dr=0.92 t=0.0ps kin=1.50 pot=19.77 Rg=16.918 SPS=2

bl=200 pos[1]=[-5.1 -11.9 -1.8] dr=0.95 t=0.0ps kin=1.51 pot=19.79 Rg=16.614 SPS=2295 
3rd loop 
---

bl=201 pos[1]=[-4.8 -11.6 -1.2] dr=0.96 t=0.0ps kin=1.50 pot=19.78 Rg=16.627 SPS=2215 
3rd loop 
---

bl=202 pos[1]=[-4.8 -11.7 -1.1] dr=0.94 t=0.0ps kin=1.47 pot=19.82 Rg=16.640 SPS=2275 
3rd loop 
---

bl=203 pos[1]=[-4.5 -11.2 -0.3] dr=0.93 t=0.0ps kin=1.51 pot=19.77 Rg=16.644 SPS=2286 
3rd loop 
---

bl=204 pos[1]=[-4.9 -10.8 -0.7] dr=0.93 t=0.0ps kin=1.48 pot=19.76 Rg=16.646 SPS=2303 
3rd loop 
---

bl=205 pos[1]=[-4.3 -10.5 0.4] dr=0.94 t=0.0ps kin=1.50 pot=19.75 Rg=16.656 SPS=2201 
3rd loop 
---

bl=206 pos[1]=[-4.9 -11.7 1.7] dr=0.95 t=0.0ps kin=1.48 pot=19.75 Rg=16.688 SPS=2253 
3rd loop 
---

bl=207 pos[1]=[-3.6 -10.4 2.3] dr=0.94 t=0.0ps kin=1.48 pot=19.77 Rg=16.694 SPS=2273 
3rd loop 
---

bl=208 pos[1]=[-4.8 -11.6 1.0] dr=0.93 t=0.0ps kin=1.47 pot=19.78 Rg=16.684 SPS=2254 
3rd loop 
---

bl=209 pos[1]=[-4.6 -11.1 0.0] dr=0.94 t=0.0ps kin=1.50 pot=19.78 Rg=16.696 SPS=2310 


bl=270 pos[1]=[-9.6 -14.5 -1.6] dr=0.95 t=0.0ps kin=1.53 pot=19.74 Rg=16.162 SPS=2197 
3rd loop 
---

bl=271 pos[1]=[-9.0 -15.3 -1.8] dr=0.94 t=0.0ps kin=1.49 pot=19.76 Rg=16.169 SPS=2255 
3rd loop 
---

bl=272 pos[1]=[-7.7 -13.8 -2.7] dr=0.94 t=0.0ps kin=1.50 pot=19.76 Rg=16.162 SPS=2300 
3rd loop 
---

bl=273 pos[1]=[-7.8 -13.8 -2.3] dr=0.95 t=0.0ps kin=1.47 pot=19.77 Rg=16.144 SPS=2288 
3rd loop 
---

bl=274 pos[1]=[-9.1 -13.7 -1.8] dr=0.94 t=0.0ps kin=1.47 pot=19.76 Rg=16.131 SPS=2195 
3rd loop 
---

bl=275 pos[1]=[-9.1 -13.6 -2.3] dr=0.95 t=0.0ps kin=1.46 pot=19.80 Rg=16.103 SPS=2298 
3rd loop 
---

bl=276 pos[1]=[-10.1 -14.2 -2.9] dr=0.95 t=0.0ps kin=1.53 pot=19.78 Rg=16.077 SPS=2166 
3rd loop 
---

bl=277 pos[1]=[-9.7 -13.7 -3.3] dr=0.96 t=0.0ps kin=1.52 pot=19.79 Rg=16.033 SPS=2263 
3rd loop 
---

bl=278 pos[1]=[-9.0 -12.9 -3.1] dr=0.95 t=0.0ps kin=1.52 pot=19.81 Rg=16.003 SPS=2243 
3rd loop 
---

bl=279 pos[1]=[-9.4 -12.4 -3.0] dr=0.94 t=0.0ps kin=1.51 pot=19.82 Rg=15.981 SPS=

bl=339 pos[1]=[-10.5 -11.6 -2.8] dr=0.94 t=0.0ps kin=1.51 pot=19.76 Rg=15.473 SPS=2253 
3rd loop 
---

bl=340 pos[1]=[-9.8 -11.1 -3.8] dr=0.94 t=0.0ps kin=1.49 pot=19.79 Rg=15.452 SPS=2278 
3rd loop 
---

bl=341 pos[1]=[-9.9 -11.8 -3.4] dr=0.96 t=0.0ps kin=1.51 pot=19.80 Rg=15.446 SPS=2268 
3rd loop 
---

bl=342 pos[1]=[-8.1 -12.4 -4.4] dr=0.95 t=0.0ps kin=1.50 pot=19.80 Rg=15.424 SPS=2315 
3rd loop 
---

bl=343 pos[1]=[-7.3 -12.4 -4.4] dr=0.95 t=0.0ps kin=1.48 pot=19.81 Rg=15.406 SPS=2307 
3rd loop 
---

bl=344 pos[1]=[-8.2 -13.4 -4.1] dr=0.95 t=0.0ps kin=1.49 pot=19.80 Rg=15.377 SPS=2263 
3rd loop 
---

bl=345 pos[1]=[-8.1 -13.6 -5.4] dr=0.96 t=0.0ps kin=1.49 pot=19.81 Rg=15.362 SPS=2308 
3rd loop 
---

bl=346 pos[1]=[-7.9 -14.3 -5.0] dr=0.95 t=0.0ps kin=1.48 pot=19.79 Rg=15.364 SPS=2311 
3rd loop 
---

bl=347 pos[1]=[-7.7 -14.2 -5.3] dr=0.95 t=0.0ps kin=1.48 pot=19.77 Rg=15.342 SPS=2246 
3rd loop 
---

bl=348 pos[1]=[-8.1 -14.1 -5.3] dr=0.95 t=0.0ps kin=1.51 pot=19.79 Rg=15.309 SPS=

...

bl=672 pos[1]=[-10.0 -9.2 1.4] dr=0.93 t=0.0ps kin=1.49 pot=19.81 Rg=14.652 SPS=2297 
3rd loop 
---

bl=673 pos[1]=[-9.5 -7.9 2.3] dr=0.96 t=0.0ps kin=1.48 pot=19.81 Rg=14.630 SPS=2191 
3rd loop 
---

bl=674 pos[1]=[-8.2 -5.9 2.6] dr=0.94 t=0.0ps kin=1.50 pot=19.77 Rg=14.599 SPS=2292 
3rd loop 
---

bl=675 pos[1]=[-7.8 -6.4 3.4] dr=0.92 t=0.0ps kin=1.50 pot=19.76 Rg=14.593 SPS=2277 
3rd loop 
---

bl=676 pos[1]=[-8.4 -6.0 3.5] dr=0.94 t=0.0ps kin=1.51 pot=19.77 Rg=14.629 SPS=2304 
3rd loop 
---

bl=677 pos[1]=[-8.8 -6.3 2.2] dr=0.94 t=0.0ps kin=1.48 pot=19.79 Rg=14.664 SPS=2212 
3rd loop 
---

bl=678 pos[1]=[-9.2 -7.1 1.8] dr=0.94 t=0.0ps kin=1.50 pot=19.78 Rg=14.667 SPS=2233 
3rd loop 
---

bl=679 pos[1]=[-8.5 -6.1 2.2] dr=0.95 t=0.0ps kin=1.50 pot=19.76 Rg=14.655 SPS=2125 
3rd loop 
---

bl=680 pos[1]=[-8.7 -5.5 3.1] dr=0.96 t=0.0ps kin=1.53 pot=19.73 Rg=14.664 SPS=2240 
3rd loop 
---

bl=681 pos[1]=[-9.8 -6.2 4.6] dr=0.94 t=0.0ps kin=1.48 pot=19.78 Rg=14.656 SPS=2271 
3rd loop 
---


bl=743 pos[1]=[-10.8 -10.3 2.2] dr=0.94 t=0.0ps kin=1.50 pot=19.74 Rg=14.686 SPS=2223 
3rd loop 
---

bl=744 pos[1]=[-10.3 -9.3 2.2] dr=0.93 t=0.0ps kin=1.47 pot=19.76 Rg=14.689 SPS=2233 
3rd loop 
---

bl=745 pos[1]=[-9.9 -9.5 3.0] dr=0.94 t=0.0ps kin=1.48 pot=19.76 Rg=14.679 SPS=2236 
3rd loop 
---

bl=746 pos[1]=[-9.2 -10.3 3.9] dr=0.93 t=0.0ps kin=1.48 pot=19.75 Rg=14.660 SPS=2297 
3rd loop 
---

bl=747 pos[1]=[-8.8 -9.1 2.6] dr=0.93 t=0.0ps kin=1.51 pot=19.77 Rg=14.672 SPS=2204 
3rd loop 
---

bl=748 pos[1]=[-8.1 -10.0 3.3] dr=0.95 t=0.0ps kin=1.50 pot=19.75 Rg=14.701 SPS=2248 
3rd loop 
---

bl=749 pos[1]=[-8.9 -8.7 2.8] dr=0.95 t=0.0ps kin=1.51 pot=19.76 Rg=14.725 SPS=2321 
3rd loop 
---


Statistics for the simulation chr10_chr11, number of particles: 5415,  number of chains: 2

Statistics for particle position
     mean position is:  [-1.60002087  0.32577506  0.50377871]   Rg =  14.724545
     median bond size is  0.963959044546287
     three shortest/longest (<10)/ bonds are 

bl=802 pos[1]=[-12.4 -9.4 2.9] dr=0.95 t=0.0ps kin=1.49 pot=19.75 Rg=14.419 SPS=2313 
3rd loop 
---

bl=803 pos[1]=[-11.9 -9.8 2.5] dr=0.94 t=0.0ps kin=1.50 pot=19.77 Rg=14.404 SPS=2191 
3rd loop 
---

bl=804 pos[1]=[-12.6 -10.5 2.6] dr=0.94 t=0.0ps kin=1.51 pot=19.73 Rg=14.376 SPS=2234 
3rd loop 
---

bl=805 pos[1]=[-12.2 -10.0 2.2] dr=0.95 t=0.0ps kin=1.47 pot=19.76 Rg=14.353 SPS=2294 
3rd loop 
---

bl=806 pos[1]=[-11.6 -9.9 2.3] dr=0.93 t=0.0ps kin=1.47 pot=19.77 Rg=14.340 SPS=2235 
3rd loop 
---

bl=807 pos[1]=[-11.5 -11.1 2.4] dr=0.92 t=0.0ps kin=1.50 pot=19.74 Rg=14.341 SPS=2285 
3rd loop 
---

bl=808 pos[1]=[-11.9 -10.7 1.1] dr=0.92 t=0.0ps kin=1.50 pot=19.75 Rg=14.354 SPS=2290 
3rd loop 
---

bl=809 pos[1]=[-11.6 -10.5 0.7] dr=0.94 t=0.0ps kin=1.50 pot=19.77 Rg=14.356 SPS=2218 
3rd loop 
---

bl=810 pos[1]=[-11.3 -10.7 1.3] dr=0.94 t=0.0ps kin=1.49 pot=19.76 Rg=14.352 SPS=2302 
3rd loop 
---

bl=811 pos[1]=[-11.5 -9.4 1.2] dr=0.94 t=0.0ps kin=1.48 pot=19.75 Rg=14.357 SPS=2319 

bl=872 pos[1]=[-12.8 -9.6 1.4] dr=0.94 t=0.0ps kin=1.49 pot=19.74 Rg=14.427 SPS=2253 
3rd loop 
---

bl=873 pos[1]=[-12.2 -9.9 1.6] dr=0.94 t=0.0ps kin=1.49 pot=19.76 Rg=14.425 SPS=2229 
3rd loop 
---

bl=874 pos[1]=[-13.3 -9.7 1.2] dr=0.93 t=0.0ps kin=1.50 pot=19.75 Rg=14.424 SPS=2304 
3rd loop 
---

bl=875 pos[1]=[-12.9 -9.4 1.9] dr=0.95 t=0.0ps kin=1.50 pot=19.77 Rg=14.396 SPS=2310 
3rd loop 
---

bl=876 pos[1]=[-12.7 -8.8 1.9] dr=0.93 t=0.0ps kin=1.50 pot=19.76 Rg=14.365 SPS=2269 
3rd loop 
---

bl=877 pos[1]=[-12.9 -9.1 0.5] dr=0.94 t=0.0ps kin=1.50 pot=19.79 Rg=14.354 SPS=2269 
3rd loop 
---

bl=878 pos[1]=[-12.5 -8.8 1.0] dr=0.94 t=0.0ps kin=1.51 pot=19.74 Rg=14.347 SPS=2266 
3rd loop 
---

bl=879 pos[1]=[-12.7 -9.4 1.8] dr=0.94 t=0.0ps kin=1.50 pot=19.76 Rg=14.349 SPS=2289 
3rd loop 
---

bl=880 pos[1]=[-12.2 -9.3 2.1] dr=0.94 t=0.0ps kin=1.49 pot=19.76 Rg=14.345 SPS=2273 
3rd loop 
---

bl=881 pos[1]=[-12.9 -9.7 1.8] dr=0.97 t=0.0ps kin=1.49 pot=19.81 Rg=14.324 SPS=2313 
3rd l

bl=942 pos[1]=[-8.4 -10.4 3.5] dr=0.95 t=0.0ps kin=1.47 pot=19.76 Rg=13.892 SPS=2327 
3rd loop 
---

bl=943 pos[1]=[-9.0 -9.7 3.8] dr=0.93 t=0.0ps kin=1.50 pot=19.75 Rg=13.934 SPS=2219 
3rd loop 
---

bl=944 pos[1]=[-9.1 -9.6 4.1] dr=0.94 t=0.0ps kin=1.50 pot=19.75 Rg=13.961 SPS=2218 
3rd loop 
---

bl=945 pos[1]=[-10.0 -10.0 4.6] dr=0.95 t=0.0ps kin=1.47 pot=19.78 Rg=13.975 SPS=2253 
3rd loop 
---

bl=946 pos[1]=[-8.5 -10.3 4.2] dr=0.95 t=0.0ps kin=1.50 pot=19.76 Rg=13.973 SPS=2208 
3rd loop 
---

bl=947 pos[1]=[-8.5 -10.0 4.7] dr=0.94 t=0.0ps kin=1.49 pot=19.78 Rg=13.955 SPS=2314 
3rd loop 
---

bl=948 pos[1]=[-7.4 -8.8 4.7] dr=0.95 t=0.0ps kin=1.52 pot=19.76 Rg=13.942 SPS=2296 
3rd loop 
---

bl=949 pos[1]=[-7.7 -8.2 4.9] dr=0.95 t=0.0ps kin=1.52 pot=19.79 Rg=13.937 SPS=2244 
3rd loop 
---


Statistics for the simulation chr10_chr11, number of particles: 5415,  number of chains: 2

Statistics for particle position
     mean position is:  [-1.31748324  0.29032331  0.21154068]   Rg = 

**Simulation Run Successfully !**  
but now we're facing the same error when using visualization tools  

![error](../img/pymol_numAtoms.png)